KEY DATES LOG:

5/7 - got next.js hello world working on custom domain with ec2
11/7 - set up the frontend server so that it autostarts the webpage when launched, and keeps the same IP address 

Previous experiment with training LoRAs weren't possible on the Mac, so just going to run a regular stable diffusion model.

Based on advice from ChatGPT and my own research:

- Next.js as it is a full package and is optimal for SEO
- FastAPI (Python) for backend IF NEEDED due to familiarity and it's a modern technology
- TailwindCSS for styling
- EC2 for running the AI model (Stable Diffusion)
- Redis for queuing
- DynamoDB for general storage NEED A SQL DATABASE TOO
- Firebase Auth + Stripe for authorisation and billing
- CloudFront for CDN (or is this already integrated into the next.js hoster)
- ComfyUI for generating images
- GitHub Actions for CI/CD
- RDS for credits data storage

Next.js setup:

npx create-next-app@latest my-ai-app
cd my-ai-app

say yes to all options

npm run dev, to run

hosting on ec2:

launch a micro ec2 instance
ssh into it (run in bash: chmod 400 /Users/klioballiu/Desktop/RollerAI/rollerai_key.pem then  ssh -i "rollerai_key.pem" ubuntu@ec2-18-175-197-226.eu-west-2.compute.amazonaws.com),
 install node and git, run the build
 curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash -
sudo apt-get install -y nodejs
sudo apt install git
git clone https://github.com/your-username/your-repo.git
cd to relevant folder...
npm install
npm run build
npm start

setup reverse proxy using nginx and domain (open relevant ports and point domain to ec2 with an A record), and secure with https using 'lets encrypt'

pm2 is used to keep the app running and doesn't 'block' the ssh line

I had the right feeling that adding 3000 to inbound rules is not sound security-wise. Instead we use reverse proxy to reflect it onto the protected ports

need to do elastic IP on the front end EC2, but I'll do it when it's needed.

[User Web App]
     |
     v
[Frontend (EC2 1) — Next.js + Auth + Stripe]
     |
     v
[Redis Queue (hosted or self-managed)]
     |
     v
[Worker Server (EC2 2) — ComfyUI/Auto1111 + Stable Diffusion]

User Request
   ↓
Web Server (e.g., Next.js + API)
   ↓
Redis Queue  ← Monitor (sees queue length or task time) + KURBENETES?
   ↓
Worker (on EC2) ← Autoscaler (spins more EC2s if needed) + KURBENETES?

+--------------------+
|  User Request/API  |
+--------------------+
          ↓
+--------------------+         +--------------------+
|   API Pod (FastAPI)|  → → →  |     Redis Pod      |
+--------------------+         +--------------------+
          ↓                            ↑
   Enqueue image job          Store job in queue
          ↓                            ↓
+---------------------+   ← ← ←  +---------------------+
|  Worker Pod (Celery,|         |  KEDA (Auto-Scaler)  |
|  RQ, custom script) |         +---------------------+
+---------------------+

1 - FastAPI endpoint 
2 - Worker script + image generator
3 - Redis queue 
4 - Deploy on Kubernetes
5 - Add KEDA + stress test & autoscale

I ASSUME THIS WILL ALL RUN IN A 'virtual private cloud'

Deciding to not do image to image as it's too complex to get working right. ControlNet is definitely needed as I tried an online img2img and even when fine-tuning the image weight the results were still rubbish.

Also choosing SD1.5 as it is the lightest model

Turns out spot instances can actually last a few hours so they can be good for images - check their 'frequency of interruption'

Using PostgreSQL for data integrity for the credits system (use amazon RDS)

Choosing 'diffusers' to run SD1.5 on the EC2 as GPT says it's the best for CLI and API pipelines.

Going to try g4n.xlarge first, then g5.xlarge if its not enough

If I understand this right, the reason we need event-driven scaling is because an EC2 instance is needed per VM due to the block-y nature of processing a text-to-image request. OR is Kubernetes used on a global and local scale - I guess it makes sense if I'm paying for the EC2 anyway I might aswell use the full processing power, so Kubernetes will scale this LOCALLY first, and then if not enough to process the queue, it will then recruit more EC2s. But, KEDA can instead know how long the queue will be and get enough EC2s ready, which is more efficient. GPT agreed.

BUT for simplicity I might not bother running Kubernetes on the actual EC2s themselves and just accept it is inefficient for now.

KEY - will definitely have to shut down all these EC2s, too many are needed in total.
I'll just have to record videos of it working, and make notes on how it all links together.

Just going to set up elastic IP for the frontend EC2 since that's the thing many other entities link to.

Not going to use Stripe as I'm already learning interaction through an external API by using Firebase Auth, and it'll be long to actually test with money. Just going to modify database entries when needed.


I'm also setting up the launch commands so when the EC2 instance starts it runs code to get the next.js server running

setting up the launch script:

cd RollerAI/my-app
npm run build
npm start

Ended up using pm2 as launch scripts require you to create the machine after defining the launch template:

pm2 start npm --name "roller-ai" -- start

pm2 save
pm2 startup

sudo env PATH=$PATH:/usr/bin /usr/lib/node_modules/pm2/bin/pm2 startup systemd -u ubuntu --hp /home/ubuntu

That works now, next is to set up another EC2 as the worker bot and have them communicate via an API.

setting up the worker bot:

- chose 'deep learning ami neuron'
- g4dn.xlarge to start with 
- had to request an increase in 'Running On-Demand G and VT instances' to 4 (It's not getting approved currently)

OKAY I tried getting approval for spot and normal g-type EC2s and it's not getting approved.
Based on GPT reccomendations I'm going to try runpod.io and GCP. I want to make sure there's still an aspect where the Redis/KEDA queue comes into use. 

Okay it's looking like I'll use runpod over GCP for simplicity, it's for short tasks so it's better, and no need for all the IAM/enterprise stuff on the google side.

Just make sure you follow security practices if you use the 'community cloud' option which is where I think the spot pricing stuff is.

Right now the planned system diagram is looking something like this:

+----------------+             +----------------+
| Firebase Auth  | <---------> |   Credits DB   |
+----------------+             +----------------+
          ^                             ^
          |                             |
          v                             v
     +---------------------------------------+
     |              Frontend VM              |
     +---------------------------------------+
          ^                        |   ▲
          |                        v   |
          |              +---------------------+
          |              |    Redis + KEDA     |
          |              +---------------------+
          |                        |
          |                        v
          |              +---------------------+
          |              |       RunPod        |
          |              |  +---------------+  |
          |              |  |     Pod 1     |  |
          |              |  +---------------+  |
          |              |  |     Pod 2     |  |
          |              |  +---------------+  |
          |              |  |     Pod 3     |  |
          |              |  +---------------+  |
          |              +---------------------+
          |                        |
          |                        v
          |              +---------------------+
          +------------- |         S3          |
                         +---------------------+

Runpod:

- need to decide which of the services I will use, I know I will use the cheapest option which is the RTX A5000.
- I will use 'Secure Cloud' not 'Community Cloud' as community just seems like an unreliable and inconsistent mess. The three options remaining are: pod spot; pod normal hourly; serverless

So I've decided to use pod spot. The jobs are very short and I want to implement the try/except logic anyway. Also I'm not choosing serverless due to price and the fact that there'd be no point for the Redis/KEDA stuff. Going to pause this stuff and get Firebase/credits working first.

Firebase + credits:

Key for SQL - have a normal id as the primary as it makes SQL operations quicker, and use 'UNIQUE NOT NULL' for unique identifiers so your database maintains integrity.

I initally thought once I get the token from Firebase it just gives me an ID I use to search in the database but you need a verifier before you allow it to access a DB, and in general it's not good practice to expose a database to the frontend. The SDK will live on the same server as the frontend server. I'll have to use pm2 to have the two systems working simultaneously.

Using Amazon RDS to host a PostgreSQL database.

ACTUALLY I realised I need to do authentication at the end because otherwise I can't test Redis.

New system diagram:


+----------------+             +----------------+
| Firebase Auth  |             |   Credits DB   |
+----------------+             +----------------+
          ^                             ^
          |                             |
          v                             v
     +---------------------------------------+
     |              Frontend VM              |
     |         --------------------          |
     |           Firebase Auth SDK           |
     +---------------------------------------+
          ^                        |   ▲
          |                        v   |
          |              +---------------------+
          |              |    Redis + KEDA     |
          |              +---------------------+
          |                        |
          |                        v
          |              +---------------------+
          |              |       RunPod        |
          |              |  +---------------+  |
          |              |  |     Pod 1     |  |
          |              |  +---------------+  |
          |              |  |     Pod 2     |  |
          |              |  +---------------+  |
          |              |  |     Pod 3     |  |
          |              |  +---------------+  |
          |              +---------------------+
          |                        |
          |                        v
          |              +---------------------+
          +------------- |         S3          |
                         +---------------------+

Ok I reiterated to GPT I just want two pods, and a Python script was suggested instead. I'm going to try figure it out myself, because I want some type of hysterisis where it spins up the second one when queue is bigger than 4 but then only stops when queue is less than 2. It's better too since it will get me to MVP quicker. Also removed the arrow between Firebase Auth.

Also I thought more about how I'll actually get the image displayed. My thought was that I generate a unique ID with the prompt, then I poll the s3 waiting for the image to be generated that has the id. But S3 URLs are random, it's costly to constantly poll s3, and there's no timeout/error strategy. 

After conversation with GPT the new strategy is runpod uploads to an s3 url, and sends a 'complete' signal back to the redis ec2. The front end is constantly polling the redis hash to check if the relevant id key value is complete, and when it is, it grabs the key which is the url, sends that url to the front end which loads the right image . 

So this means the arrow between runpod and the redis VM goes both ways now. Going to upgrade to a t3.medium just in case.

New system diagram is below:

+----------------+          +----------------+
| Firebase Auth  |          |   Credits DB   |
+----------------+          +----------------+
          ^                          ^
          |                          |
          v                          v
+---------------------------------------+
|              Frontend VM              |
|         --------------------          |
|           Firebase Auth SDK      (EC2)|
+---------------------------------------+
     ^                        |   ^
     |                        v   |
     |    +------------------------------+
     |    | Redis + Python script   (EC2)|
     |    +------------------------------+
     |                        |  ^
     |                        v  |
     |              +---------------------+
     |              |       RunPod        |
     v              |  +---------------+  |
+------------+      |  |     Pod 1     |  |
|     S3     |<-----|  +---------------+  |
+------------+      |  |     Pod 2     |  |
                    |  +---------------+  |
                    +---------------------+



This is the work sequence I'm going to follow, I'm pretty sure that my system diagram is set in stone now:

- set up t3.medium for redis system with ElasticIP
- work on getting string input with text box that sends to redis backend side
- then get the API working so runpod sends to s3
- create redis queue so multiple requests can happen at the same time
- create redis hash and setup APIs and frontend code so image appears on frontend
- add on your python script so tasks are balanced between two workers spun up as necessary with hysterisis
- set up firebase auth so user has to log in before being able to request an image
- add a credits system linked to RDS

So the python pseudo-equivalent of pm2 is uvicorn:

sudo nano /etc/systemd/system/fastapi.service
inside it write this

[Unit]
Description=FastAPI app with Uvicorn
After=network.target

[Service]
User=ubuntu
WorkingDirectory=/home/ubuntu/RollerAI/redis_vm
ExecStart=/usr/bin/env uvicorn scaler:app --host 0.0.0.0 --port 8000
Restart=always

[Install]
WantedBy=multi-user.target

sudo systemctl daemon-reexec
sudo systemctl daemon-reload
sudo systemctl enable fastapi.service
sudo systemctl start fastapi.service

adding CORS to try get communication working - this wasn't the issue. GPT suggested using console on chrome, I see it's because although the frontend is in https, backened is http, so I get 'mixed content' error